In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import xskillscore as xs
import xesmf as xe
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
import intake
import util 

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

/Users/manuelhernandez/opt/miniconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
col_dict = {}
col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")

import pprint
uni_dict = col.unique(['source_id','experiment_id','table_id'])

In [3]:
# Choose how much to coarsen data
coarsen_size = 2

# Define the common target grid axes
dlon, dlat = 1., 1.
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90.+dlat/2., 90., dlat)),
                     'lon': (['lon'], np.arange(0.+dlon/2., 360., dlon)),})

# Regridding function
def regrid_to_common(ds, ds_out):
    """
    Regrid from rectilinear grid to common grid
    """
    regridder = xe.Regridder(ds, ds_out, 'bilinear',periodic=True, reuse_weights=True)
    return regridder(ds)

Rearth = 6.378E6   # radius of Earth in meters
# a DataArray that gives grid cell areas on the lat/lon grid (in units of m^2)
area = (np.deg2rad(dlat)*Rearth) * (np.deg2rad(dlon)*Rearth*np.cos(np.deg2rad(ds_out.lat))) * xr.ones_like(ds_out.lon)

# coarsen
area = area.coarsen({'lat': coarsen_size, 'lon': coarsen_size}, boundary='exact').mean()

# month lengths
days_in_month = np.array([31 , 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
days_in_month = xr.DataArray(days_in_month, coords=[np.arange(1,13,1)], dims=['month'])

In [4]:
models = set(uni_dict['source_id']['values']) # all the models
for experiment_id in ['historical', 'ssp585','ssp370']:
    query = dict(experiment_id=experiment_id, table_id='Amon',
                 variable_id='va', grid_label='gn')  
    cat = col.search(**query)
    models = models.intersection({model for model in cat.df.source_id.unique().tolist()})
models = list(models)
models
cat = col.search(experiment_id=['historical'], table_id='Amon',
                 variable_id='va', grid_label='gn', source_id=models)   
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': False})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 11 group(s)


In [15]:
time_slice = slice('1950','2014')
plev_slice = 85000
lat_bnds  = [27.5,42.5]
lon_bnds  = [257.5,270]
region = dict(lat=slice(27.5,42.5),lon=slice(257.5,270))

In [16]:
ds_dict = {}
va_dict = {}
for name, ds in tqdm(dset_dict.items()):
    if ('latitude' in ds.dims) and ('longitude'in ds.dims):
        ds = ds.rename({'latitude':'lat', 'longitude':'lon'}) #Some Models Labeled Differently
    ds = xr.decode_cf(ds)
    ds = (ds.mean(dim='member_id'))
    ds = ds.sel(plev=plev_slice)
    
    # drop redundant variables (like "height: 2m")
    for coord in ds.coords:
        if coord not in ['lat','lon','time']:
            ds = ds.drop(coord)
            
            
    va_clim = ds.groupby('time.month').mean('time')
    va_anom = ds.groupby('time.month') - va_clim
    va_anom = va_anom - va_anom.sel(time=slice('1970','2000')).mean(dim='time')
    ds_con = va_anom.sel(**region)
    
    
    
    ## Calculate global-mean meridional wind
    #cos_lat_2d = np.cos(np.deg2rad(ds['lat'])) * xr.ones_like(ds['lon']) # effective area weights
    #va = (
    #    (ds['va'] * cos_lat_2d).sum(dim=['lat','lon']) /
    #    cos_lat_2d.sum(dim=['lat','lon'])
    #    )
    
    #va_dict[name] = va.squeeze()
    #ds_dict[name] = ds
   # monClim = ds.sel(time=time_slice).groupby('time.month').mean(dim='time')
    

    #ds = (ds.sel(time=time_slice).groupby('time.month').mean(dim='time') -
    #        ds.sel(time=time_slice).mean(dim='time'))

#    ds_new = regrid_to_common(clim['va'],ds_out)

    # Maybe chance this at pre-processing stage?
    #ds.attrs['name'] = ds.attrs['institution']
    
    #Add Ensemble as a New Dimension
    #ds = ds.expand_dims({'ensemble': np.array([ds.attrs['name']])},0)
               

    # We should keep the metadata!
#   ds_new = ds_new.coarsen({'lat':coarsen_size, 'lon':coarsen_size}, boundary='exact').mean()   

    # Add this to the dictionary
    #ds_dict[name] = ds

/Users/manuelhernandez/opt/miniconda3/lib/python3.7/site-packages/xarray/core/indexing.py:1311: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]
/Users/manuelhernandez/opt/miniconda3/lib/python3.7/site-packages/xarray/core/indexing.py:1311: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]
/Users/manuelhernandez/opt/miniconda3/lib/python3.7/site-packages/xarray/core/indexing.py:1311: PerformanceWarning: Slicing with an out-of-order index is generating 167 times more chunks
  return self.array[key]
/Users/manuelhernandez/opt/miniconda3/lib/python3.7/site-packages/xarray/core/indexing.py:1311: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]
/Users/manuelhernandez/opt/miniconda3/lib/python3.7/site-packages/xarray/core/indexing.py:1311: PerformanceWarning: Slicing with an out-of-order index i

/Users/manuelhernandez/opt/miniconda3/lib/python3.7/site-packages/xarray/core/indexing.py:1311: PerformanceWarning: Slicing with an out-of-order index is generating 165 times more chunks
  return self.array[key]


In [17]:
ds_con

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 7, lon: 4, time: 1980)
Coordinates:
  * lon        (lon) float64 258.8 262.5 266.2 270.0
  * lat        (lat) float64 27.95 30.19 32.42 34.66 36.89 39.13 41.37
  * time       (time) object 1850-01-17 00:00:00 ... 2014-12-17 00:00:00
    month      (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: bnds
Data variables:
    areacella  (time, lat, lon) float64 dask.array<chunksize=(1, 7, 4), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(1, 7, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(1, 4, 2), meta=np.ndarray>
    va         (time, lat, lon) float32 dask.array<chunksize=(1, 7, 4), meta=np.ndarray>